In [ ]:
import nltk
from transformers import pipeline
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
import string

In [ ]:
# Download necessary NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Load BART model for abstractive summarization
abstractive_summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Abstractive summary using transformers
def abstractive_summary(text, max_len=130, min_len=30):
    summary = abstractive_summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
    return summary[0]['summary_text']

In [ ]:
# Extractive summary using NLTK
def extractive_summary(text, num_sentences=3):
    stop_words = set(stopwords.words("english"))
    words = nltk.word_tokenize(text.lower())

    freq_table = defaultdict(int)
    for word in words:
        if word in stop_words or word in string.punctuation:
            continue
        freq_table[word] += 1

    sentences = sent_tokenize(text)
    sentence_scores = {}

    for sent in sentences:
        word_count = 0
        for word in nltk.word_tokenize(sent.lower()):
            if word in freq_table:
                word_count += 1
                sentence_scores[sent] = sentence_scores.get(sent, 0) + freq_table[word]
        if word_count > 0:
            sentence_scores[sent] = sentence_scores[sent] / word_count

    # Select top N sentences
    summarized_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:num_sentences]
    return " ".join(summarized_sentences)

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Assume you've fine-tuned the model
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Save model and tokenizer to folder
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/merges.txt',
 './saved_model/added_tokens.json')

In [9]:
from google.colab import files

# Example: download the 'saved_model/my_model.pb' file
files.download('saved_model/model.safetensors')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>